In [130]:
import json
import pandas as pd

# Load the JSON data from the file
with open('exercises.json', 'r') as f:
    data = json.load(f)

name, force, level, equipment, primary_muscles, secondary_muscles, category = [], [], [], [], [], [], []
equipment_filter = ['barbell', 'dumbbell', 'machine', "e-z curl bar", "cable"]

# Iterate over each muscle group and each exercise within that group
for muscle_group in data:
    for exercise in data[muscle_group]:
        if exercise.get('equipment') in equipment_filter:
            name.append(exercise['name'])
            level.append(exercise.get('level', 'N/A'))  # Assuming 'level' corresponds to 'difficulty'
            equipment.append(exercise.get('equipment', 'N/A'))
            primary_muscles.append(exercise.get('primaryMuscles'))
            secondary_muscles.append(exercise.get('secondaryMuscles'))
            category.append(exercise.get('category'))

# Create a DataFrame
df = pd.DataFrame({'name': name, 'level': level, 'equipment': equipment, 'primary_muscles': primary_muscles, 'secondary_muscles': secondary_muscles, 'category': category})
df.head()

,name,level,equipment,primary_muscles,secondary_muscles,category
0,Ab Crunch Machine,intermediate,machine,[abdominals],[],strength
1,Alternate Hammer Curl,beginner,dumbbell,[biceps],[forearms],strength
2,Alternate Incline Dumbbell Curl,beginner,dumbbell,[biceps],[forearms],strength
3,Alternating Cable Shoulder Press,beginner,cable,[shoulders],[triceps],strength
4,Alternating Deltoid Raise,beginner,dumbbell,[shoulders],[],strength


In [138]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df['features'] = df.apply(lambda x: ' '.join(x['primary_muscles']) + ' ' + x['category'], axis=1)

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the features
tfidf_matrix = vectorizer.fit_transform(df['features'])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def get_recommendations(name, selected_exercises=[], cosine_sim=cosine_sim):
    # Get the index of the exercise
    idx = df[df['name'] == name].index[0]

    # Get the pairwise similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the exercises based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar exercises
    sim_scores = sim_scores[1:3]

    # Get the exercise indices
    exercise_indices = [i[0] for i in sim_scores]

    exercise_indices = [idx for idx in exercise_indices if df['name'].iloc[idx] not in selected_exercises]

    # Return the top 5 most similar exercises
    return df['name'].iloc[exercise_indices]

# Example usage
print("Recommendations for 'Pullups':")
print(get_recommendations('Barbell Curl'))

Recommendations for 'Pullups':
2     Alternate Incline Dumbbell Curl
11                       Barbell Curl
Name: name, dtype: object


In [139]:
push_muscles = ['chest', 'shoulders', 'triceps']
pull_muscles = ['middle back', 'biceps', 'forearms', 'lower back', 'lats', 'traps']
legs_muscles = ['quadriceps', 'hamstrings', 'calves', 'glutes']



In [141]:
def select_exercises_from_muscle_group(muscles, num_exercises_per_muscle=2):
    strength_df = df[df['category'] == 'strength']
    selected_exercises = []
    for muscle in muscles:
        # Filter the DataFrame for exercises targeting the current muscle
        filtered_df = strength_df[strength_df['primary_muscles'].apply(lambda x: muscle in x)].copy()
        
        # Select the specified number of random exercises for the current muscle
        exercises = filtered_df.sample(n=num_exercises_per_muscle)['name'].tolist()
        selected_exercises.extend(exercises)
    
    return selected_exercises

# Select two exercises for each muscle group in 'push_muscles'
selected_exercises = select_exercises_from_muscle_group(push_muscles, num_exercises_per_muscle=2)

# Get recommendations for each selected exercise
recommendations = []
for exercise in selected_exercises:
    recommendations.extend(get_recommendations(exercise, recommendations))

print("Selected Push Exercises:")
for exercise in selected_exercises:
    print("-", exercise)
print("\nRecommendations:")
for exercise in recommendations:
    print("-", exercise)


selected_exercises = select_exercises_from_muscle_group(pull_muscles, num_exercises_per_muscle=1)

# Get recommendations for each selected exercise
recommendations = []
for exercise in selected_exercises:
    recommendations.extend(get_recommendations(exercise, recommendations))

print("\nSelected Pull Exercises:")
for exercise in selected_exercises:
    print("-", exercise)
print("\nRecommendations:")
for exercise in recommendations:
    print("-", exercise)


selected_exercises = select_exercises_from_muscle_group(legs_muscles, num_exercises_per_muscle=1)

# Get recommendations for each selected exercise
recommendations = []
for exercise in selected_exercises:
    recommendations.extend(get_recommendations(exercise, recommendations))

print("\nSelected Leg Exercises:")
for exercise in selected_exercises:
    print("-", exercise)
print("\nRecommendations:")
for exercise in recommendations:
    print("-", exercise)

Selected Push Exercises:
- Butterfly
- Cable Crossover
- Cable Shoulder Press
- External Rotation
- Triceps Pushdown - Rope Attachment
- Low Cable Triceps Extension

Recommendations:
- Barbell Bench Press - Medium Grip
- Barbell Guillotine Bench Press
- Alternating Deltoid Raise
- Anti-Gravity Press
- Cable Lying Triceps Extension
- Cable One Arm Tricep Extension

Selected Pull Exercises:
- Straight Bar Bench Mid Rows
- Concentration Curls
- Palms-Down Dumbbell Wrist Curl Over A Bench
- Barbell Deadlift
- One Arm Lat Pulldown
- Cable Shrugs

Recommendations:
- Bent Over One-Arm Long Bar Row
- Bent Over Two-Arm Long Bar Row
- Alternate Incline Dumbbell Curl
- Barbell Curl
- Dumbbell Lying Pronation
- Dumbbell Lying Supination
- Stiff Leg Barbell Good Morning
- Deadlift with Bands
- Cable Incline Pushdown
- Close-Grip Front Lat Pulldown
- Barbell Shrug Behind The Back
- Cable Shrugs

Selected Leg Exercises:
- Jefferson Squats
- Lying Leg Curls
- Dumbbell Seated One-Leg Calf Raise
- One-L